### Plan of Attack <br>
#### Web-scrape signals from the git repo
#### Also webscrape daily bitcoin price action
#### Scale and Standardize important features
#### Build out the EMA, DEMA and TEMA technical signals 
#### For aa 7 day forward looking model, change target value to be 7 days after the current date (sounds like a rotate/shift approach for this use-case

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import quandl
import investpy

In [8]:
def webscraping_helper(URL,col_name,join_df,join=True,check_column=True,check_URL = True,clear_URL_array=False,show_details=False):
    '''This function scraps the given link and returns dataframe
    __________
    Parameters:
        URL(string): URL to be scrapped from bitcoin website
        col_name(string): column name for dataframe
        join_df(variable)= dataframe withwhich output dataframe will be left joined on Date
        join(boolean)= iF True,join, else don't join
        check_column(boolean)= check if column name already exists
        check_URL(boolean)= check if URL is already processed
        clear_URL_array(boolean)= if true URL_processed array will be cleared
        show_details(boolean)= various details wil be printed such as scrapping first and last details, df head & df tail     
        '''
        
    print(f'processing {col_name}')

    #clear URL append array
    if clear_URL_array==True:
        URL_array.clear()

    #set join parameters if false
    if join == False:
        join_df = None
        check_column=False

    #process column name by making it lowercase and replacing spaces,commas, full stops
    col_name = col_name.lower().replace(',','').replace(" ", "_").replace(".", "_")

    #col_name validation if exists already
    if check_column==True and col_name in list(join_df.columns):
        print(f'column {col_name} already esists in dataframe, stopped here')
        return join_df

    #web scrapping
    page = requests.get(URL)
    soup = page.content
    soup = str(soup)
    scraped_output = (soup.split('[[')[1]).split('{labels')[0][0:-2]
    if show_details == True:
        print('head')
        print({scraped_output[0:30]})
        print('tail')
        print({scraped_output[-30:]})

    processed_str = scraped_output.replace('new Date(','')
    processed_str = processed_str.replace(')','')
    processed_str = processed_str.replace('[','')
    processed_str = processed_str.replace(']','')
    processed_str = processed_str.replace('"','')

    processed_str_list = processed_str.split(',')
    date_list,data_list = processed_str_list[::2],processed_str_list[1::2]

    #validate column lengths
    if len(date_list)!=len(data_list):
        print(f'date & data length:{len(date_list),len(data_list),len(date_list)==len(data_list)}')
        
    if col_name not in list(join_df.columns) and join == True:
        df = pd.DataFrame()
        df['Date'] = pd.to_datetime(date_list)
        df[col_name] = data_list
        join_df = pd.merge(join_df,df,on=['Date'],how='left')
        if show_details == True:
            print('*'*100)
            print('df head')
            print(df.head(1))
            print('*'*100)
            print('df tail')
            print(df.tail(1))
            print('*'*100)
            print(f'output df shape= {df.shape},joined_df shape = {join_df.shape}')
            print('='*100)
            print(f'Number of duplicate columns in dataframe {df.columns.duplicated().sum()}')
            print('='*100)
    
        return join_df

In [11]:
btc_df = investpy.get_crypto_historical_data(crypto='bitcoin',from_date='01/01/2013',to_date='09/02/2022')
btc_df = btc_df.reset_index()
btc_df.drop(['Currency','Volume'],inplace=True,axis=1)
btc_df.columns = ['Date','opening_price','highest_price','lowest_price','closing_price']
btc_df.head()

,Date,opening_price,highest_price,lowest_price,closing_price
0,2013-01-01,13.5,13.6,13.2,13.3
1,2013-01-02,13.3,13.4,13.2,13.3
2,2013-01-03,13.3,13.5,13.3,13.4
3,2013-01-04,13.4,13.5,13.3,13.5
4,2013-01-05,13.5,13.6,13.3,13.4


In [12]:
from collections import defaultdict
scraping_lookup = defaultdict()

In [13]:
scraping_lookup['avg block size'] = 'https://bitinfocharts.com/comparison/size-btc.html'
scraping_lookup['sent by adress'] = 'https://bitinfocharts.com/comparison/sentbyaddress-btc.html'
scraping_lookup['avg mining difficulty'] = 'https://bitinfocharts.com/comparison/bitcoin-difficulty.html'
scraping_lookup['avg hashrate'] = 'https://bitinfocharts.com/comparison/bitcoin-hashrate.html'
scraping_lookup['mining profitability'] = 'https://bitinfocharts.com/comparison/bitcoin-mining_profitability.html'
scraping_lookup['Sent coins in USD'] = 'https://bitinfocharts.com/comparison/sentinusd-btc.html'
scraping_lookup['avg transaction fees'] = 'https://bitinfocharts.com/comparison/bitcoin-transactionfees.html'
scraping_lookup['median transaction fees'] = 'https://bitinfocharts.com/comparison/bitcoin-median_transaction_fee.html'
scraping_lookup['avg block time'] = 'https://bitinfocharts.com/comparison/bitcoin-confirmationtime.html'
scraping_lookup['avg transaction value'] = 'https://bitinfocharts.com/comparison/transactionvalue-btc.html'
scraping_lookup['median transaction value'] = 'https://bitinfocharts.com/comparison/mediantransactionvalue-btc.html'
scraping_lookup['tweets'] = 'https://bitinfocharts.com/comparison/tweets-btc.html'
scraping_lookup['google trends'] = 'https://bitinfocharts.com/comparison/google_trends-btc.html'
scraping_lookup['active addresses'] = 'https://bitinfocharts.com/comparison/activeaddresses-btc.html'
scraping_lookup['top100 to total percentage'] = 'https://bitinfocharts.com/comparison/top100cap-btc.html'
scraping_lookup['avg fee to reward'] = 'https://bitinfocharts.com/comparison/fee_to_reward-btc.html'

In [14]:
scraping_lookup

defaultdict(None,
            {'avg block size': 'https://bitinfocharts.com/comparison/size-btc.html',
             'sent by adress': 'https://bitinfocharts.com/comparison/sentbyaddress-btc.html',
             'avg mining difficulty': 'https://bitinfocharts.com/comparison/bitcoin-difficulty.html',
             'avg hashrate': 'https://bitinfocharts.com/comparison/bitcoin-hashrate.html',
             'mining profitability': 'https://bitinfocharts.com/comparison/bitcoin-mining_profitability.html',
             'Sent coins in USD': 'https://bitinfocharts.com/comparison/sentinusd-btc.html',
             'avg transaction fees': 'https://bitinfocharts.com/comparison/bitcoin-transactionfees.html',
             'median transaction fees': 'https://bitinfocharts.com/comparison/bitcoin-median_transaction_fee.html',
             'avg block time': 'https://bitinfocharts.com/comparison/bitcoin-confirmationtime.html',
             'avg transaction value': 'https://bitinfocharts.com/comparison/transac

In [15]:
for scraping_name in scraping_lookup:
    btc_df = webscraping_helper(scraping_lookup[scraping_name], scraping_name, join_df=btc_df, join=True)

processing avg block size
processing sent by adress
processing avg mining difficulty
processing avg hashrate
processing mining profitability
processing Sent coins in USD
processing avg transaction fees
processing median transaction fees
processing avg block time
processing avg transaction value
processing median transaction value
processing tweets
processing google trends
processing active addresses
processing top100 to total percentage
processing avg fee to reward


In [16]:
btc_df.head()

,Date,opening_price,highest_price,lowest_price,closing_price,avg_block_size,sent_by_adress,avg_mining_difficulty,avg_hashrate,mining_profitability,...,avg_transaction_fees,median_transaction_fees,avg_block_time,avg_transaction_value,median_transaction_value,tweets,google_trends,active_addresses,top100_to_total_percentage,avg_fee_to_reward
0,2013-01-01,13.5,13.6,13.2,13.3,89033,26174,2979637,24331539528899,2246,...,0.0107,0.0067,8.889,625.432,14.518,NaN,1.194,37846,19.536,0.627
1,2013-01-02,13.3,13.4,13.2,13.3,114077,31809,2979637,22804647966378,2233,...,0.0108,0.0066,9.412,650.617,14.514,NaN,1.497,43104,19.597,0.835
2,2013-01-03,13.3,13.5,13.3,13.4,108023,38197,2979637,23724885599725,2276,...,0.0118,0.00661,8.889,542.73,19.732,NaN,1.798,51268,19.621,0.925
3,2013-01-04,13.4,13.5,13.3,13.5,141811,34990,2979637,22608181137438,2259,...,0.0105,0.00661,9.412,632.431,11.384,NaN,1.841,47341,19.54,1
4,2013-01-05,13.5,13.6,13.3,13.4,118240,38008,2979637,22590695489434,2074,...,0.0105,0.00659,10.213,697.556,13.945,NaN,1.826,53417,19.543,0.885


In [17]:
btc_circulation_df = quandl.get("BCHAIN/TOTBC",authtoken='9ztFCcK4_e1xGo_gjzK7')
btc_circulation_df = btc_circulation_df.rename(columns={'Value': 'number_of_coins_in_circulation'})

In [18]:
miner_revenue_df = quandl.get("BCHAIN/MIREV",authtoken='9ztFCcK4_e1xGo_gjzK7')
miner_revenue_df = miner_revenue_df.rename(columns={'Value': 'miner_revenue'})

In [19]:
btc_onchain_df = pd.merge(btc_df,btc_circulation_df,on=['Date'],how='left')
btc_onchain_df = pd.merge(btc_df,miner_revenue_df,on=['Date'],how='left')
btc_onchain_df.head()

,Date,opening_price,highest_price,lowest_price,closing_price,avg_block_size,sent_by_adress,avg_mining_difficulty,avg_hashrate,mining_profitability,...,median_transaction_fees,avg_block_time,avg_transaction_value,median_transaction_value,tweets,google_trends,active_addresses,top100_to_total_percentage,avg_fee_to_reward,miner_revenue
0,2013-01-01,13.5,13.6,13.2,13.3,89033,26174,2979637,24331539528899,2246,...,0.0067,8.889,625.432,14.518,NaN,1.194,37846,19.536,0.627,52648.600000
1,2013-01-02,13.3,13.4,13.2,13.3,114077,31809,2979637,22804647966378,2233,...,0.0066,9.412,650.617,14.514,NaN,1.497,43104,19.597,0.835,54865.251543
2,2013-01-03,13.3,13.5,13.3,13.4,108023,38197,2979637,23724885599725,2276,...,0.00661,8.889,542.73,19.732,NaN,1.798,51268,19.621,0.925,48118.330620
3,2013-01-04,13.4,13.5,13.3,13.5,141811,34990,2979637,22608181137438,2259,...,0.00661,9.412,632.431,11.384,NaN,1.841,47341,19.54,1,50872.740000
4,2013-01-05,13.5,13.6,13.3,13.4,118240,38008,2979637,22590695489434,2074,...,0.00659,10.213,697.556,13.945,NaN,1.826,53417,19.543,0.885,51396.725494


In [21]:
btc_onchain_df['next_week_closing_price'] = btc_onchain_df['closing_price'].shift(-7)

In [22]:
missing_values = pd.DataFrame(btc_onchain_df.isna().sum(),columns=['missing_count'])
missing_values.sort_values(by='missing_count',ascending=False)

,missing_count
tweets,463
next_week_closing_price,7
google_trends,5
median_transaction_fees,0
miner_revenue,0
avg_fee_to_reward,0
top100_to_total_percentage,0
active_addresses,0
median_transaction_value,0
avg_transaction_value,0
